In [ ]:
from ngsolve import *
from draw import Draw
from netgen.geom2d import CSG2d, Circle, Rectangle

In [ ]:
circle_large = Circle( center=(0,0), radius=1.0, mat="outer", bc="bc_outer" )
circle_inner = Circle( center=(0,0), radius=0.5, mat="inner", bc="bc_inner" )
ring = circle_large - circle_inner
geo = CSG2d()
geo.Add(circle_inner)
geo.Add(ring)

mesh = Mesh(geo.GenerateMesh(maxh=0.2))
mesh.Curve(2)
fes = H1(mesh, order=2, dirichlet="bc_outer")
cf = mesh.RegionCF(VOL, dict(inner=0, outer=1))
Draw(cf, mesh)

In [ ]:
r = sqrt(x*x+y*y)
alpha = mesh.RegionCF(VOL, dict(inner=1, outer=2))
u_exact = mesh.RegionCF(VOL, dict(inner=-0.25*r**2-1/16*(log(0.5)-1), outer=-1/16*log(r)))
coef_f = mesh.RegionCF(VOL, dict(inner=1, outer=0))
Draw(u_exact,mesh)

In [ ]:
gfu = GridFunction(fes)  # solution 
u, v = fes.TnT()

In [ ]:
a = BilinearForm(fes, symmetric=True)
a += alpha * grad(u)*grad(v)*dx

f = LinearForm(fes)
f += coef_f*v*dx

In [ ]:
a.Assemble()
f.Assemble()

In [ ]:
gfu.vec.data =     a.mat.Inverse(freedofs=fes.FreeDofs(),inverse="sparsecholesky") * f.vec

In [ ]:
Draw(gfu, mesh, "gfu")